In [25]:
# -*- coding: utf-8 -*-

import os
import json
import pika
import logging
import pymongo
import pandas as pd
from itertools import chain
from datetime import datetime, timedelta
from dotenv import load_dotenv
from azure.storage.blob import BlobClient
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.enums import TA_CENTER
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

In [26]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

load_dotenv()

True

In [27]:
# Realizando conexão com o banco
if os.getenv("MONGO_CONN_STR") is None:
    logging.error("Unable to get environment variables.")
    # exit()

client = pymongo.MongoClient(
    os.getenv("MONGO_CONN_STR"), serverSelectionTimeoutMS=5000)

try:
    client.server_info()
except Exception:
    logging.error("Unable to connect to Mongo server.")
    # exit()

database = client.get_database()

In [28]:
# Extraindo coleções
cart_collection = database.get_collection("carts")
demand_collection = database.get_collection("demands")


In [29]:
pd.DataFrame(list(demand_collection.find()))

,_id,name,start_date,end_date,product_ids,state,created_on
0,66302e7e763d3d7a42c1645b,Demanda teste,2024-04-29 00:00:00,2024-04-30 00:00:00,[66302d85763d3d7a42c1643b],DRAFT,2024-04-29 23:34:22.723
1,66302ed5763d3d7a42c16476,Demanda teste 2,2024-05-02 00:00:00,2024-05-12 00:00:00,[],DRAFT,2024-04-29 23:35:49.405
2,6634eec3763d3d7a42c1681e,Teste hidráulico,2024-05-03 00:00:00,2024-05-04 00:00:00,[6634ee42763d3d7a42c1680f],DRAFT,2024-05-03 14:03:47.272
3,63c16b2ed2ebd1993366d538,demanda 01-2,2023-01-01 03:00:00,2025-01-01 03:00:00,[],NaN,NaT
4,66364835763d3d7a42c169a4,teste de data de abertura,2024-05-02 00:00:00,2024-05-05 00:00:00,"[66302d85763d3d7a42c1643b, 66344c93763d3d7a42c...",DRAFT,2024-05-04 14:37:41.832


In [30]:
pd.DataFrame(list(cart_collection.find())).demand_name.value_counts()

Carrinhoooooooo                 2
Demanda para Testar Carrinho    2
Teste Entrega                   2
Demanda teste                   2
Qualquer                        2
Carrinho Postman                1
[Carlos] Demanda Teste          1
Demanda Domingo                 1
Elétrico 2023                   1
Aaa                             1
Difuso                          1
Novo                            1
Love                            1
portugal                        1
todos                           1
TESTE CISAB                     1
Name: demand_name, dtype: int64

In [31]:
# Extraindo demandas que fecham na data de hoje
today = datetime.today()
tomorrow = today + timedelta(days=1)

today = today.replace(hour=0, minute=0, second=0, microsecond=0)
tomorrow = tomorrow.replace(hour=0, minute=0, second=0, microsecond=0)

demands = demand_collection.find({
    "end_date": {"$gte": today, "$lt": tomorrow}
})

closed_demands = [str(demand["_id"]) for demand in demands]
if len(closed_demands) == 0:
    logging.info("There's no demand closing today!")
    # exit()

In [32]:
# Extraindo carrinhos fechados das demandas fechadas
carts = cart_collection.find({
    "state" : { "$eq" : "closed"},
    "demand_id": { "$in": closed_demands}
  })

In [33]:
# Transformando as coleções em dataframes
df_carts = pd.DataFrame(list(carts))

logging.info(str(len(df_carts)) + " orders for today.")

2024-06-27 10:33:03 INFO 1 orders for today.


In [34]:
# Pré-processamento
df_carts = df_carts.rename(columns={'_id': 'cart_id'})

In [35]:
# Expandindo a coluna de produtos
df_exploded = df_carts.explode(column="products")

In [36]:
# Extraindo atributos dos produtos e colocando-os em outras colunas
def getAtrr(row, atrribute):
  rowDict = dict(row)
  return rowDict[atrribute]


In [37]:
df_exploded["product_id"] = df_exploded["products"].apply(lambda x: getAtrr(x, "_id"))
df_exploded["quantity"] = df_exploded["products"].apply(lambda x: getAtrr(x, "quantity"))

In [38]:
# Montando o nome final do produto e adicionando-o na coluna descrição
def getProductDesc(row):
  measurementsList = list(row["products"]["measurements"])
  newList = []
  for m in measurementsList:
    newList.append(" ".join(list(m.values())))

  normsList = list(row["products"]["norms"])
  
  return row["products"]["name"] + " " + " ".join(newList) + " " + " ".join(normsList)

In [39]:
df_exploded["description"] = df_exploded.apply(lambda x: getProductDesc(x), axis=1)

In [40]:
# Exportando o dataframe final para planilha em Excel na Azure
def uploadToStorage(file_name):
    try:
        blob = BlobClient.from_connection_string(conn_str=os.getenv(
            "AZURE_CONN_STR"), container_name="cisab-consolidados", blob_name=file_name)
    except:
        logging.error("Unable to connect with Blob Storage.")
        # exit()

    try:
        with open(file_name, "rb") as data:
            blob.upload_blob(data)
    except:
        logging.error("Unable to upload the file to Blob Storage.")
        # exit()

In [41]:
# Enviando evento para o RabbitMQ mandar o email
def createAndSendEvent(connection, spreadsheet, pdf):
    channel = connection.channel()

    to = os.getenv("RABBITMQ_TO")
    event = {
        "pattern": "send_email",
        "data": {
        "message": {
            "to": to,
            "subject": "Consolidado de pedidos",
            "body": "A demanda fechou e você pode baixar o consolidado de pedidos pelo link: <a href='" + os.getenv("AZURE_BLOB_STORAGE") + spreadsheet + "'>clique aqui</a>. Você também pode baixar o relatório de pedidos pelo link: <a href='" + os.getenv("AZURE_BLOB_STORAGE") + pdf + "'>clique aqui</a>"
        }
        }
    }

    channel.queue_declare(queue='notifier')
    channel.basic_publish(exchange='',
                        routing_key='notifier',
                        body=json.dumps(event, ensure_ascii=False))

    logging.info(f"Email has been sent to {to}")

    channel.close()
    
def sendEmail(spreadsheet, pdf):
    try:
        credentials = pika.PlainCredentials(os.getenv("RABBITMQ_USER"),
                                            os.getenv("RABBITMQ_PASSWORD"))
        connection = pika.BlockingConnection(
            pika.ConnectionParameters(os.getenv("RABBITMQ_CONN_STR"),
                                    int(os.getenv("RABBITMQ_PORT")),
                                    '/',
                                    credentials))
        
        createAndSendEvent(connection, spreadsheet, pdf)

    except:
        logging.error("Unable to connect with RabbitMQ.")
        # exit()

In [42]:
# Gerando relatório em PDF com os pedidos de cada município
def createPDF(df, file_name):
  county_prods = []
  for county in df["Município / Autarquia"].unique():
    prods_list = df[df["Município / Autarquia"] == county][["Produto", "Quantidade"]].values.tolist()
    prods_list.insert(0, [county])
    county_prods.append(prods_list)

  subheaders = []
  unnested_list = list(chain(*county_prods))
  for county in df["Município / Autarquia"].unique():
    subheaders.append(unnested_list.index([county]))

  # Criando PDF
  doc = SimpleDocTemplate(file_name, pagesize=letter)

  # Colocando título
  styles = getSampleStyleSheet()
  styles.add(ParagraphStyle(name='centered', parent=styles['Heading3'], alignment=TA_CENTER))
  title = Paragraph("Relatório de pedidos por município", styles["Title"])
  subtitle = Paragraph(df.loc[0, 'demand_name'], styles["centered"])

  # Criando tabela
  table = Table(unnested_list)

  for i in subheaders:
    # Adicionando estilo para os subtítulos (nome dos municípios)
    table.setStyle(TableStyle([
        ("BACKGROUND", (0, i), (-1, i), "gray"),
        ("TEXTCOLOR", (0, i), (-1, i), "white"),
    ]))

  # Gerando PDF
  doc.build([title, subtitle, table])

In [43]:
# Gerando a planilha a partir do dataframe final
def generateFiles(df, demand_id):
    df_demand = df[df.demand_id == demand_id].reset_index()

    if(len(df_demand) > 0):
        df_pivot = df_demand.pivot_table(index="Produto", values="Quantidade", columns="Município / Autarquia")
        df_pivot["Total"] = df_pivot.sum(axis=1)
        df_pivot = df_pivot.sort_values(by="Produto")

        format_data = "%Y-%m-%d %H-%M-%S"
        file_name = datetime.strftime(datetime.today(), format_data) + \
            " " + df_demand.loc[0, "demand_name"]
        sheet_name = f"{file_name}.xlsx"
        pdf_name = f"{file_name}.pdf"

        df_pivot.to_excel(sheet_name)
        createPDF(df_demand, pdf_name)

        uploadToStorage(sheet_name)
        uploadToStorage(pdf_name)
        sendEmail(sheet_name, pdf_name)
    else:
        logging.warning("There weren't any orders for " + demand_id)

In [44]:
# Pós-processamento
newNames = {"county_name": "Município / Autarquia", "description": "Produto", "quantity": "Quantidade"}
df = df_exploded.rename(columns=newNames)

In [45]:
df

,cart_id,state,products,user_id,updated_on,product_ids,demand_name,demand_id,user_name,county_id,Município / Autarquia,product_id,Quantidade,Produto
0,66302f1c763d3d7a42c1649f,closed,"{'_id': '66302d85763d3d7a42c1643b', 'name': 'P...",66302987763d3d7a42c163cd,2024-04-29 23:37:00.876,"[{'product_id': '66302d85763d3d7a42c1643b', 'q...",Demanda teste,66302e7e763d3d7a42c1645b,Daniela Gomes,66302987763d3d7a42c163ca,Conselheiro Lafaiete,66302d85763d3d7a42c1643b,5,Produto Teste Altura 5 dm Largura 3 cm NBR5 PBR3


In [46]:
# Executando processos finais para cada demanda fechada no dia
for demand in closed_demands:
    generateFiles(df, demand)

2024-06-27 10:33:08 INFO Request URL: 'https://catalogv2.blob.core.windows.net/cisab-consolidados/2024-06-27%2010-33-08%20Demanda%20teste.xlsx'
Request method: 'PUT'
Request headers:
    'Content-Length': '4980'
    'x-ms-blob-type': 'REDACTED'
    'If-None-Match': '*'
    'x-ms-version': 'REDACTED'
    'Content-Type': 'application/octet-stream'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.14.1 Python/3.10.5 (Windows-10-10.0.19045-SP0)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': 'cad5c3a4-3489-11ef-9175-3052cb8340ee'
    'Authorization': 'REDACTED'
A body is sent with the request
2024-06-27 10:33:09 INFO Response status: 201
Response headers:
    'Content-Length': '0'
    'Content-MD5': 'REDACTED'
    'Last-Modified': 'Thu, 27 Jun 2024 13:33:09 GMT'
    'ETag': '"0x8DC96ADAF8FB778"'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '894d664e-501e-003a-6596-c841d2000000'
    'x-ms-client-request-id': 'ca